In [1]:
# Imports
import numpy as np
import scipy.io as sio
from sklearn import decomposition
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.pipeline import FeatureUnion
from sklearn import cross_validation
import autosklearn
from sklearn.feature_selection import f_classif, SelectPercentile

In [2]:
# Import the data
trainData = sio.loadmat('Train.mat')
testData = sio.loadmat('Test.mat')

# Get the values of the train data
Xtrain = trainData.get('Xtrain')
Ytrain = trainData.get('Ytrain')

In [4]:
# Change the data
# Set up the classifier
# Use PCA to reduce the dimensionalit
comp =453 # number of components
cw = {}
cw[1] = 1
cw[0] = 1
cw[3] = 1
pca = decomposition.PCA(n_components=comp)

selection = SelectPercentile(f_classif, percentile=4)

combined_features = FeatureUnion([ ("univ_select", selection)])

In [5]:
# Run feature selection
Xtrain = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True).fit_transform(Xtrain)
X_pca = combined_features.fit(Xtrain, np.ravel(Ytrain)).transform(Xtrain)

In [6]:
# Set up classifier
automl = autosklearn.AutoSklearnClassifier()

In [7]:
automl.fit(X_pca, Ytrain)

[INFO] [2015-11-24 03:05:22,098:AutoML(1):894685adab9c31371fb5e315b3713987] Start calculating metafeatures for 894685adab9c31371fb5e315b3713987
[INFO] [2015-11-24 03:05:22,122:AutoML(1):894685adab9c31371fb5e315b3713987] Calculating Metafeatures (categorical attributes) took  0.02
[INFO] [2015-11-24 03:05:22,314:AutoML(1):894685adab9c31371fb5e315b3713987] Calculating Metafeatures (encoded attributes) took  0.19sec
[INFO] [2015-11-24 03:05:23,673:AutoML(1):894685adab9c31371fb5e315b3713987] Time left for 894685adab9c31371fb5e315b3713987 after finding initial configurations: 3596.03sec
[INFO] [2015-11-24 03:05:23,675:autosklearn.util.smac] Start SMAC with 3596.03sec time left
[INFO] [2015-11-24 03:05:23,675:autosklearn.util.smac] Calling: smac --numRun 1 --scenario /tmp/autosklearn_tmp_1909_1535/894685adab9c31371fb5e315b3713987.scenario --num-ei-random 1000 --num-challengers 100 --initial-incumbent DEFAULT --retryTargetAlgorithmRunCount 0 --intensification-percentage 0.5 --validation false

/home/cole/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1136: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)
/home/cole/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


<AutoSklearnClassifier(AutoSklearnClassifier-1, initial)>

In [8]:
# Get the test data
testX = testData.get('Xtest')
# Use PCA on the test set
testX = preprocessing.StandardScaler(copy=True, with_mean=True, with_std=True).fit_transform(testX)
testNew =  combined_features.transform(testX)

In [9]:
# Get the results from the classifier
testY = automl.predict( testNew )

In [10]:
# Get zero or one probabilities
new = np.zeros((1001,3))

for i in xrange(0, 1001):
    if( testY[i,0] == max(testY[i,:])):
        new[i,:] = [1.0000, 0.0000, 0.0000]
    elif( testY[i,1] == max(testY[i,:])):
        new[i,:] = [0.0000, 1.0000, 0.0000]
    else:
        new[i,:] = [0.0000, 0.0000, 1.0000]

In [12]:
# Put into a csv file
np.savetxt('prediction.csv', new, delimiter=",")